In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Avance clásico (feedforward) y flujo de control (circuitos dinámicos)

import Tabs from '@theme/Tabs';
import TabItem from '@theme/TabItem';

# Avance clásico (feedforward) y flujo de control
{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Versiones de paquetes</b></summary>

El código de esta página se desarrolló utilizando los siguientes requisitos.
Recomendamos usar estas versiones o más recientes.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
> **Note:** La nueva versión de los circuitos dinámicos está ahora disponible para todos los usuarios en todos los backends. Ahora puedes ejecutar circuitos dinámicos a escala de utilidad (utility scale). Consulta [el anuncio](/announcements/product-updates/2025-09-25-new-dynamic-circuits) para obtener más detalles.

Los circuitos dinámicos son herramientas poderosas con las que puedes medir qubits en la mitad de la ejecución de un circuito cuántico y luego realizar operaciones lógicas clásicas dentro del circuito, basadas en el resultado de esas mediciones intermedias del circuito (mid-circuit measurements). Este proceso también se conoce como _avance clásico (classical feedforward)_. Si bien aún nos encontramos en una etapa temprana para entender cómo conseguir la mejor manera de aprovechar los circuitos dinámicos, la comunidad de investigación cuántica ya ha identificado una serie de casos de uso (use cases), tales como los siguientes:

* Preparación eficiente de estados cuánticos, tales como el [estado GHZ,](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.5.030339) el [estado W,](https://arxiv.org/abs/2403.07604) (para obtener más detalles sobre el estado W, comunícate e infórmate por igual refiriéndote hacia ["State preparation by shallow circuits using feed forward"](https://arxiv.org/abs/2307.14840)) y una amplia clase de [estados de producto matricial (matrix product states)](https://arxiv.org/abs/2404.16083)
* [Entrelazamiento de largo alcance eficiente](https://journals.aps.org/prxquantum/abstract/10.1103/PRXQuantum.5.030339) entre qubits interactuando situados en el mismo chip usando circuitos poco profundos (shallow circuits).
* [Muestreo eficiente de circuitos tipo-IQP](https://arxiv.org/pdf/2505.04705).

Estas mejoras introducidas por los circuitos dinámicos, sin embargo, vienen con ciertas compensaciones (trade-offs). Las mediciones intermedias del circuito y las operaciones lógicas clásicas típicamente tienen tiempos de ejecución más largos que las puertas de dos qubits, y este incremento de tiempo podría anular los beneficios de reducir la profundidad del circuito. Por tanto, disminuir la longitud o tiempo de las mediciones de circuito intermedio, se ha posicionado como un área de enfoque para las mejoras a medida que IBM Quantum&reg; lanza la [nueva versión](/announcements/product-updates/2025-03-03-new-version-dynamic-circuits) de circuitos dinámicos.

La [especificación de OpenQASM 3](https://openqasm.com/language/classical.html#looping-and-branching) define varias estructuras de flujo de control, pero Qiskit Runtime actualmente solo admite la declaración condicional `if`. En el SDK de Qiskit, esto corresponde al método [if_test](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#if_test) en [QuantumCircuit.](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) Este método devuelve un [gestor de contexto (context manager)](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers) y normalmente se usa en una declaración `with`. Esta guía describe cómo usar esta declaración condicional.

> **Note:** Los ejemplos de código en esta guía utilizan la instrucción de medición estándar para mediciones en medio del circuito. Sin embargo, se recomienda que en su lugar utilices la instrucción [`MidCircuitMeasure`](/guides/measure-qubits#midcircuit), si el backend la admite. Consulta la documentación sobre [Mediciones a la mitad del circuito](/guides/measure-qubits#mid-circuit-measurements) para obtener más detalles.

## Declaración `if`
La declaración `if` se utiliza para realizar operaciones de manera condicional basándose en el valor de un bit o registro clásico.

En el siguiente ejemplo, aplicamos una puerta de Hadamard a un qubit y lo medimos. Si el resultado es 1, entonces aplicamos una puerta X en el qubit, lo cual tiene el efecto de invertirlo (flipping it) de regreso al estado 0. Luego medimos el qubit nuevamente. El resultado de medición (resulting measurement outcome) debería ser 0 con una probabilidad del 100%.

In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister

qubits = QuantumRegister(1)
clbits = ClassicalRegister(1)
circuit = QuantumCircuit(qubits, clbits)
(q0,) = qubits
(c0,) = clbits

circuit.h(q0)
# Use MidCircuitMeasure() if it's supported by the backend.
# circuit.append(MidCircuitMeasure(), [q0], [c0])
circuit.measure(q0, c0)
with circuit.if_test((c0, 1)):
    circuit.x(q0)
circuit.measure(q0, c0)
circuit.draw("mpl")

# example output counts: {'0': 1024}

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/60924bfa-50ed-4d9d-a17b-9d64f2cc053f-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/60924bfa-50ed-4d9d-a17b-9d64f2cc053f-0.svg)

A la declaración `with` se le puede dar un objetivo de asignación que es en sí mismo un gestor de contexto (context manager) que puede ser almacenado y posteriormente utilizado para crear un bloque `else`, el cual se ejecuta siempre que los contenidos del bloque `if` *no* se ejecutan.

En el ejemplo a continuación, inicializamos registros con dos qubits y dos bits clásicos. Aplicamos una puerta de Hadamard al primer qubit y lo medimos. Si el resultado es 1, entonces aplicamos una puerta de Hadamard sobre el segundo qubit; de no ser así (o caso contrario, otherwise), aplicamos una puerta X sobre el segundo qubit. Finalmente medimos también el segundo qubit.

In [2]:
qubits = QuantumRegister(2)
clbits = ClassicalRegister(2)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1) = qubits
(c0, c1) = clbits

circuit.h(q0)
circuit.measure(q0, c0)
with circuit.if_test((c0, 1)) as else_:
    circuit.h(q1)
with else_:
    circuit.x(q1)
circuit.measure(q1, c1)

circuit.draw("mpl")

# example output counts: {'01': 260, '11': 272, '10': 492}

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/20f0640a-a3f7-41b3-aada-b66bc89b0555-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/20f0640a-a3f7-41b3-aada-b66bc89b0555-0.svg)

Además del condicionamiento sobre un solo bit clásico, también es posible condicionar sobre el valor de un registro clásico compuesto por múltiples bits.

En el ejemplo a continuación, aplicamos puertas de Hadamard a dos qubits y los medimos. Si el resultado es `01`, es decir, el primer qubit es 1 y el segundo qubit es 0, entonces aplicamos una puerta X a un tercer qubit. Finalmente, medimos el tercer qubit. Ten en cuenta que, para mayor claridad, elegimos especificar el estado del tercer bit clásico, que es 0, en la condición `if`. En el dibujo del circuito, la condición se indica mediante los círculos sobre los bits clásicos que están siendo condicionados. Un círculo negro indica condicionamiento para 1, mientras que un círculo blanco indica condicionamiento para 0.

In [3]:
qubits = QuantumRegister(3)
clbits = ClassicalRegister(3)
circuit = QuantumCircuit(qubits, clbits)
(q0, q1, q2) = qubits
(c0, c1, c2) = clbits

circuit.h([q0, q1])
circuit.measure(q0, c0)
circuit.measure(q1, c1)
with circuit.if_test((clbits, 0b001)):
    circuit.x(q2)
circuit.measure(q2, c2)

circuit.draw("mpl")

# example output counts: {'101': 269, '011': 260, '000': 252, '010': 243}

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/98e8f552-4169-42a3-8182-e14e9ffb59e2-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/98e8f552-4169-42a3-8182-e14e9ffb59e2-0.svg)

## Expresiones clásicas
El módulo de expresiones clásicas de Qiskit [`qiskit.circuit.classical`](https://docs.quantum.ibm.com/api/qiskit/circuit_classical) contiene una representación de exploración de las operaciones de ejecución (runtime operations) en valores clásicos durante la ejecución de circuitos. Debido a limitaciones de hardware, actualmente solo se admiten condiciones en `QuantumCircuit.if_test()`.

El siguiente ejemplo muestra que puedes usar el cálculo de la paridad para crear un estado GHZ de n-qubits usando circuitos dinámicos. Primero, se generan $n/2$ pares de Bell en los qubits adyacentes. Luego, se unen (o enlazan) estos pares mediante el uso de una capa de puertas CNOT entre ellos. A continuación, mides el qubit objetivo (target qubit) de todos los CNOT anteriores y reinicias cada qubit medido al estado $\vert 0 \rangle$. Aplicas la operación $X$ a cada sitio no medido para el cual la paridad de todos los bits de medición precedentes sea impar. Finalmente, se aplican puertas CNOT a los qubits medidos para restablecer el entrelazamiento (entanglement) perdido en la medición.

En el cálculo de paridad, el primer elemento de la expresión construida involucra la elevación (lifting) del objeto de Python `mr[0]` a un nodo [`Value`](https://docs.quantum.ibm.com/api/qiskit/circuit_classical#value) (`lift` se utiliza para convertir objetos arbitrarios en expresiones clásicas). Esto no es necesario para `mr[1]` y el posible registro clásico posterior, ya que son entradas para `expr.bit_xor`, y cualquier elevación necesaria se realiza automáticamente en estos casos. Tales expresiones se pueden construir en bucles y otras construcciones (constructs).

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.circuit.classical import expr

num_qubits = 8
if num_qubits % 2 or num_qubits < 4:
    raise ValueError("num_qubits must be an even integer ≥ 4")
meas_qubits = list(range(2, num_qubits, 2))  # qubits to measure and reset

qr = QuantumRegister(num_qubits, "qr")
mr = ClassicalRegister(len(meas_qubits), "m")
qc = QuantumCircuit(qr, mr)

# Create local Bell pairs
qc.reset(qr)
qc.h(qr[::2])
for ctrl in range(0, num_qubits, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

# Glue neighboring pairs
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

# Measure boundary qubits between pairs,reset to 0
for k, q in enumerate(meas_qubits):
    qc.measure(qr[q], mr[k])
    qc.reset(qr[q])

# Parity-conditioned X corrections
# Each non-measured qubit gets flipped iff the parity (XOR) of all
# preceding measurement bits is 1
for tgt in range(num_qubits):
    if tgt in meas_qubits:  # skip measured qubits
        continue
    # all measurement registers whose physical qubit index < tgt
    left_bits = [k for k, q in enumerate(meas_qubits) if q < tgt]
    if not left_bits:  # skip if list empty
        continue

    # build XOR-parity expression
    parity = expr.lift(
        mr[left_bits[0]]
    )  # lift the first bit to Value so it will be treated like a boolean.
    for k in left_bits[1:]:
        parity = expr.bit_xor(
            mr[k], parity
        )  # calculate parity with all other bits
    with qc.if_test(parity):  # Add X if parity is 1
        qc.x(qr[tgt])

# Re-entangle measured qubits
for ctrl in range(1, num_qubits - 1, 2):
    qc.cx(qr[ctrl], qr[ctrl + 1])

In [5]:
qc.draw(output="mpl", style="iqp", idle_wires=False, fold=-1)

<Image src="../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/d0f0abdb-50d5-408d-a704-a1a555acdd85-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/classical-feedforward-and-control-flow/extracted-outputs/d0f0abdb-50d5-408d-a704-a1a555acdd85-0.svg)

## Encontrar backends que admiten circuitos dinámicos
Para encontrar todos los backends a los que tu cuenta puede acceder y que admiten circuitos dinámicos, ejecuta un código como el siguiente. Este ejemplo asume que has [guardado tus credenciales de inicio de sesión](/guides/save-credentials). También podrías [especificar explícitamente las credenciales](/guides/initialize-account#explicit) al inicializar tu cuenta de servicio de Qiskit Runtime. Esto te permitiría ver los backends disponibles en una instancia específica o tipo de plan, por ejemplo.

> **Note:** - Los backends que están disponibles para la cuenta dependen de la instancia especificada en las credenciales.
> - La nueva versión de los circuitos dinámicos está ahora disponible para todos los usuarios en todos los backends. Consulta [el anuncio](/announcements/product-updates/2025-09-25-new-dynamic-circuits) para obtener más detalles.

In [6]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
dc_backends = service.backends(dynamic_circuits=True)
print(dc_backends)

[<IBMBackend('ibm_pittsburgh')>, <IBMBackend('ibm_boston')>, <IBMBackend('ibm_fez')>, <IBMBackend('ibm_miami')>, <IBMBackend('ibm_marrakesh')>, <IBMBackend('ibm_torino')>, <IBMBackend('ibm_kingston')>]


## Qiskit Runtime limitations

Be aware of the following constraints when running dynamic circuits in Qiskit Runtime.

- Due to the limited physical memory on control electronics, there is also a limit on the number of `if` statements and size of their operands. This limit is a function of the number of broadcasts and the number of broadcasted bits in a job (not a circuit).

   When processing an `if` condition, measurement data needs to be transferred to the control logic to make that evaluation. A broadcast is a transfer of unique classical data, and broadcasted bits is the number of classical bits being transferred. Consider the following:

   ```python
   c0 = ClassicalRegister(3)
   c1 = ClassicalRegister(5)
   ...
   with circuit.if_test((c0, 1)) ...
   with circuit.if_test((c0, 3)) ...
   with circuit.if_test((c1[2], 1)) ...
   ```
   In the previous code example, the first two `if_test` objects on `c0` are considered one broadcast because the content of `c0` did not change, and thus does not need to be re-broadcasted. The `if_test` on `c1` is a second broadcast. The first one broadcasts all three bits in `c0` and the second broadcasts just one bit, making a total of four broadcasted bits.

   Currently, if you broadcast 60 bits each time, then the job can have approximately 300 broadcasts. If you broadcast just one bit each time, however, then the job can have 2400 broadcasts.

- The operand used in an `if_test` statement must be 32 or fewer bits. Thus, if you are comparing an entire `ClassicalRegister`, the size of that `ClassicalRegister` must be 32 or fewer bits. If you are comparing just a single bit from a `ClassicalRegister`, however, that `ClassicalRegister` can be of any size (since the operand is only one bit).

   For example, the "Not valid" code block does not work because `cr` is more than 32 bits.  You can, however, use a classical register wider than 32 bits if you are testing only one bit, as shown in the "Valid" code block.

   <Tabs>
   <TabItem value="Not valid" label="Not valid">
      ```python
         cr = ClassicalRegister(50)
         qr = QuantumRegister(50)
         circuit = QuantumCircuit(qr, cr)
         ...
         circ.measure(qr, cr)
         with circ.if_test((cr, 15)):
            ...
      ```
   </TabItem>
   <TabItem value="Valid" label="Valid">
      ```python
         cr = ClassicalRegister(50)
         qr = QuantumRegister(50)
         circuit = QuantumCircuit(qr, cr)
         ...
         circ.measure(qr, cr)
         with circ.if_test((cr[5], 1)):
            ...
      ```
   </TabItem>
   </Tabs>

- Nested conditionals are not allowed. For example, the following code block will not work because it has an `if_test` inside another `if_test`:
   <Tabs>
    <TabItem value="Not valid" label="Not valid">
        ```python
           c1 = ClassicalRegister(1, "c1")
           c2 = ClassicalRegister(2, "c2")
           ...
           with circ.if_test((c1, 1)):
            with circ.if_test(c2, 1)):
             ...
        ```
     </TabItem>
     <TabItem value="Valid" label="Valid">
        ```python
        cr = ClassicalRegister(2)
        ...
        with circuit.if_test((cr, 0b11)):
          ...
        ```
     </TabItem>
    </Tabs>

- Having `reset` or measurements inside conditionals is not supported.
- Arithmetic operations are not supported.
- See the [OpenQASM 3 feature table](/docs/guides/qasm-feature-table) to determine which OpenQASM 3 features are supported on Qiskit and Qiskit Runtime.
- When OpenQASM 3 (instead of `QuantumCircuit`) is used as the input format to pass circuits to Qiskit Runtime primitives, only instructions that can be loaded into Qiskit are supported. Classical operations, for example, are not supported because they cannot be loaded into Qiskit. See [Import an OpenQASM 3 program into Qiskit](/docs/guides/interoperate-qiskit-qasm3#import-an-openqasm-3-program-into-qiskit) for more information.
- The `for`, `while`, and `switch` instructions are not supported.

## Use dynamic circuits with Estimator

Since Estimator does not support dynamic circuits, you can use Sampler and build your own measurement circuits instead. Alternatively, you can use the [Executor primitive,](/docs/guides/directed-execution-model#executor-primitive) which supports dynamic circuits.

To replicate Estimator's behavior, follow this process:

1. Group the terms of all observables into a partition.  This can be done by using the [`PauliList` API,](/docs/api/qiskit/qiskit.quantum_info.PauliList#group_qubit_wise_commuting) for example.
     <Admonition type="note">
      You can use the [`BitArray`](https://quantum.cloud.ibm.com/docs/en/api/qiskit/qiskit.primitives.BitArray#expectation_values) primitive attribute to compute the expectation values of the provided observables.
     </Admonition>
2. Execute one basis change circuit per partition (whichever basis change needs to be done for each partition). See the Measurement bases addon utility  [`measurement_bases` module](https://github.com/Qiskit/qiskit-addon-utils/blob/38ea05431f2e9830adf4ec9265f6d19758a32096/qiskit_addon_utils/exp_vals/measurement_bases.py) for more information. [Get started with utilities.](/docs/guides/qiskit-addons-utils#get-started-with-utilities)
3. Add back together the results for each partition.

## Next steps

<Admonition type="tip" title="Recommendations">
- Learn how to implement accurate dynamic decoupling by using [stretch.](/docs/guides/stretch)
- Learn about the shorter [mid-circuit measurements](/docs/guides/measure-qubits#mid-circuit-measurements) that reduce the circuit time.
- Use [circuit schedule visualization](/docs/guides/visualize-circuit-timing#qiskit-runtime-support) to debug and optimize your dynamic circuits.
</Admonition>